In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
element_size = 28
time_steps = 28
num_classes = 10
batch_size = 128
hidden_layer_size = 128

LOG_DIR = "logs/RNN_with_summaries"

In [6]:
_inputs = tf.placeholder(tf.float32,shape=[None, time_steps,element_size],name='inputs')
y = tf.placeholder(tf.float32, shape=[None, num_classes],name='inputs')

In [7]:
# TensorFlow built-in functions
rnn_cell = tf.contrib.rnn.BasicRNNCell(hidden_layer_size)
outputs, _ = tf.nn.dynamic_rnn(rnn_cell, _inputs, dtype=tf.float32)

Wl = tf.Variable(tf.truncated_normal([hidden_layer_size, num_classes],mean=0,stddev=.01))
bl = tf.Variable(tf.truncated_normal([num_classes],mean=0,stddev=.01))

In [8]:
def get_linear_layer(vector):
    return tf.matmul(vector, Wl) + bl

In [9]:
last_rnn_output = outputs[:,-1,:]
final_output = get_linear_layer(last_rnn_output)
softmax = tf.nn.softmax_cross_entropy_with_logits(logits=final_output,labels=y)
cross_entropy = tf.reduce_mean(softmax)
train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(final_output,1))
accuracy = (tf.reduce_mean(tf.cast(correct_prediction, tf.float32)))*100

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [10]:
sess=tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

test_data = mnist.test.images[:batch_size].reshape((-1,time_steps, element_size))
test_label = mnist.test.labels[:batch_size]

for i in range(3001):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    batch_x = batch_x.reshape((batch_size, time_steps, element_size))
    sess.run(train_step,feed_dict={_inputs:batch_x,y:batch_y})
    
    if i % 1000 == 0:
        acc = sess.run(accuracy, feed_dict={_inputs: batch_x,y: batch_y})
        loss = sess.run(cross_entropy,feed_dict={_inputs:batch_x,y:batch_y})
        print ("Iter " + str(i) + ", Minibatch Loss= " + \
               "{:.6f}".format(loss) + ", Training Accuracy= " + \
               "{:.5f}".format(acc))
    
print ("Testing Accuracy:",sess.run(accuracy, feed_dict={_inputs: test_data, y: test_label}))

Iter 0, Minibatch Loss= 2.298859, Training Accuracy= 11.71875
Iter 1000, Minibatch Loss= 0.330717, Training Accuracy= 88.28125
Iter 2000, Minibatch Loss= 0.197839, Training Accuracy= 94.53125
Iter 3000, Minibatch Loss= 0.033289, Training Accuracy= 99.21875
Testing Accuracy: 98.4375
